## Table of Contents
* Loading & Preprocessing
* No Synthetic Data
    * Neural Network with No Synthetic Data
* GAN Model
    * Neural Network with GAN Data 
* Diffusion Model
    * Neural Network with Diffusion Model Data
* Results Summary

## Loading & Preprocessing

In [ ]:
import pandas as pd
import numpy as np
import matplotlib as mpl
import tensorflow as tf
import keras
import scipy
import torch
import random
from sklearn.preprocessing import LabelEncoder, MinMaxScaler, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from matplotlib import pyplot
from numpy import zeros, ones
from numpy.random import randn, randint
from keras.models import Sequential, Model, load_model
from keras.layers import Dense, Reshape, LeakyReLU, Dropout, GaussianNoise, BatchNormalization, Activation
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from scipy.stats import gmean
from scipy.spatial.distance import jensenshannon
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F

In [ ]:
# set random seed to 15 throughout notebook
seed = 15
random.seed(seed)
np.random.seed(seed)
tf.random.set_seed(seed)
torch.manual_seed(seed)

In [ ]:
# import raw data from UCI's "Adult" dataset of 1994 census data
raw_data1 = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data',header=None)
raw_data2 = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.test',header=None,skiprows=1)
raw_data = pd.concat([raw_data1,raw_data2])

# add headers
headers = ['age','workclass','fnlwgt','education','education-num','marital-status','occupation','relationship','race','sex','capital-gain','capital-loss','hours-per-week','native-country','target']
data = raw_data.to_csv('adult_headers.csv',header=headers)
data = pd.read_csv('adult_headers.csv')

# clean data
data = data.iloc[: , 1:]
data['target'] = data['target'].replace([' <=50K.',' >50K.'],[' <=50K',' >50K'])
data.head()

In [ ]:
# encode categorical data
class MultiColumnLabelEncoder:
    def __init__(self,columns = None):
        self.columns = columns

    def fit(self,X,y=None):
        return self

    def transform(self,X):
        output = X.copy()
        if self.columns is not None:
            for col in self.columns:
                output[col] = LabelEncoder().fit_transform(output[col])
        else:
            for colname,col in output.iteritems():
                output[colname] = LabelEncoder().fit_transform(col)
        return output

    def fit_transform(self,X,y=None):
        return self.fit(X,y).transform(X)

data = MultiColumnLabelEncoder(columns = ['workclass','education','marital-status','occupation','relationship','race','sex','native-country','target']).fit_transform(data)
data.head()

In [ ]:
# split target variable
X=data.iloc[: , :-1]
y=data.iloc[: , -1:]

# downsample to ~10,000 records (note that random states are set to 15 in all cases)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.8, random_state=15)
X_oos=X_test
y_oos=y_test
X_experiment = X_train
y_experiment = y_train

# split downsampled data into test & train
X_train, X_test, y_train, y_test = train_test_split(X_experiment, y_experiment, test_size=0.3, random_state=15)
print ('X_train:')
print(X_train.head())

## No Synthetic Data

### Neural Network With No Synthetic Data

In [ ]:
# build model
nn_model = Sequential()
nn_model.add(Dense(14, activation='LeakyReLU', input_dim=14))
nn_model.add(Dense(32, activation='LeakyReLU'))
nn_model.add(Dense(32, activation='LeakyReLU'))
nn_model.add(Dense(1, activation='sigmoid'))
nn_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['AUC'])

# set parameters
early_stopping_raw = EarlyStopping(monitor='val_loss', mode='min', patience=400, verbose=1)
model_save_raw = ModelCheckpoint('nn_raw_best_model.h5', save_best_only=True, monitor='val_auc', mode='max', verbose=1)

#train model             
nn_model_no_synthetic = nn_model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=2000, batch_size=32, verbose=1, callbacks=[early_stopping_raw, model_save_raw])

In [ ]:
# plot learning curves
pyplot.plot(nn_model_no_synthetic.history['auc'], label='train')
pyplot.plot(nn_model_no_synthetic.history['val_auc'], label='test')
pyplot.legend()

In [ ]:
# load best model
nn_raw_model = load_model('nn_raw_best_model.h5')

# evaluate model on test data (how model would be evaluated at the time)
predictions = nn_raw_model.predict(X_test)
nn_raw_model_auc_test = roc_auc_score(y_test, predictions)
print("Neural Net Raw Model AUC on Test Data: " + str(round(nn_raw_model_auc_test,4)))

# evaluate model on out of sample data (simulating a production environment)
predictions = nn_raw_model.predict(X_oos)
nn_raw_model_auc_oos = roc_auc_score(y_oos, predictions)
print("Neural Net Raw Model AUC on Out of Sample Data: " + str(round(nn_raw_model_auc_oos,4)))

## GAN Model

In [ ]:
# normalize training data between -1 and 1
scaler = MinMaxScaler(feature_range=(-1, 1))
X_train_adj = scaler.fit_transform(X_train)  

# define generator
def build_generator(latent_dim):
    model = Sequential()
    model.add(GaussianNoise(0.1, seed=15, input_shape=(latent_dim,)))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dense(32))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dense(32))
    model.add(BatchNormalization(momentum=0.8))
    model.add(Dense(latent_dim, activation='tanh'))
    model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['AUC'])
    return model
    
# define discriminator
def build_discriminator(latent_dim):
    model = Sequential()
    model.add(Dense(14, input_dim=latent_dim))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(0.2))
    model.add(Dense(32))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(0.2))
    model.add(Dense(32))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(0.2))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['AUC'])
    return model

# define GAN
def define_gan(generator, discriminator):
    discriminator.trainable = False
    model = Sequential()
    model.add(generator)
    model.add(discriminator)
    model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['AUC'])
    return model

# generate real data points
def generate_real_samples(X_train_adj, half_batch):
    ix = randint(0, X_train_adj.shape[0], half_batch) 
    X = X_train_adj[ix]  
    y = ones((half_batch, 1)) 
    return X, y

# generate latent data points
def generate_latent_points(latent_dim, n_batch):
    x_input = randn(latent_dim * n_batch)  
    z_input = x_input.reshape(n_batch, latent_dim)
    return z_input

# generate fake data points
def generate_fake_samples(generator, latent_dim, half_batch):
    z_input = generate_latent_points(latent_dim, half_batch)
    x_synthetic = generator.predict(z_input)  
    y = zeros((half_batch, 1))
    return x_synthetic, y

# save model 
def save_model(generator, discriminator, n_epochs):
    generator_filename = 'generator_model_%03d.h5' % (n_epochs+1)
    generator.save(generator_filename)
    discriminator_filename = 'discriminator_model_%03d.h5' % (n_epochs+1)
    discriminator.save(discriminator_filename)

# plot results
def plot_history(d1_hist, d2_hist, g_hist):
    pyplot.plot(d1_hist, label='d-real')
    pyplot.plot(d2_hist, label='d-fake')
    pyplot.plot(g_hist, label='gen')
    pyplot.legend()
    pyplot.savefig('gan_history_plot.png')
    pyplot.close()    

# define model training
def train(generator, discriminator, gan_model, X_train_adj, latent_dim, n_epochs, n_batch, half_batch):
    batch_per_epoch = int(X_train_adj.shape[0] / n_batch)
    n_steps = batch_per_epoch * n_epochs
    d1_hist, d2_hist, g_hist = list(), list(), list()
    for t in range(n_epochs):
      for i in range(batch_per_epoch):
        X_real, y_real = generate_real_samples(X_train_adj, half_batch)
        d_loss_r, auc_r = discriminator.train_on_batch(X_real, y_real)
        X_fake, y_fake = generate_fake_samples(generator, latent_dim, half_batch)
        d_loss_f, auc_f = discriminator.train_on_batch(X_fake, y_fake)
        x_gan = generate_latent_points(latent_dim, n_batch) 
        y_gan = ones((n_batch, 1)) 
        g_loss, g_auc = gan_model.train_on_batch(x_gan, y_gan)
        d1_hist.append(d_loss_r)
        d2_hist.append(d_loss_f)
        g_hist.append(g_loss)
      if (t+1) % 5 == 0:
        save_model(generator, discriminator, t)
        print(f"epoch {t+1}/{n_epochs}, D loss on real {round(d_loss_r,4)}, D loss on fake {round(d_loss_f,4)}, G loss {round(g_loss,4)}") 
    plot_history(d1_hist, d2_hist, g_hist)    
    
# set hyperparameters
opt = Adam(learning_rate=0.0002, beta_1=0.5)
latent_dim = 14
n_epochs = 100
n_batch = 32
half_batch = int(n_batch / 2)
generator = build_generator(latent_dim)
discriminator = build_discriminator(latent_dim)
gan_model = define_gan(generator, discriminator)

# train the model
gan_trained = train(generator, discriminator, gan_model, X_train_adj, latent_dim, n_epochs, n_batch, half_batch)

In [ ]:
# calculate JS scores for GAN generated data and identify best GAN model iteration
gan_model_js_scores = {}
latent_points = generate_latent_points(latent_dim, len(X_train))

def compile_gan_model_js_scores(n_steps, latent_points):
    for i in range(n_epochs):
        if (i+1) % 5 == 0: 
           
            # generate synthetic data from generator
            generator_saved_file = 'generator_model_%03d.h5' % (i+1)
            generator_model_trained = load_model(generator_saved_file)
            _X_synthetic_gan = generator_model_trained.predict(latent_points)
            _X_synthetic_gan = scaler.inverse_transform(_X_synthetic_gan)
            _X_synthetic_gan = np.round(_X_synthetic_gan)
                        
            # calculate JS scores and append
            synthetic_train_JS_score_gan = scipy.spatial.distance.jensenshannon(_X_synthetic_gan, X_train, base=2)
            gan_model_js_scores[i] = round(gmean(synthetic_train_JS_score_gan),4)

compile_gan_model_js_scores(n_steps, latent_points)

# return best model iteration
min_value = min(gan_model_js_scores.values())
min_key = min(gan_model_js_scores, key=gan_model_js_scores.get)+1
print(f"Best Model Iteration: {min_key}, Avg JS Score: {min_value}")

# return best 10 model iterartions
def top_10_gan_models():
    print("Top 10 GAN Model Iterations")
    top_10_gan_models = sorted(gan_model_js_scores, key=gan_model_js_scores.get, reverse=False)[:10]
    for i in range(10):
        key = top_10_gan_models[i]
        value = gan_model_js_scores[key]
        print(f"Model Iteration: {key+1}, Avg JS Score: {value}")

top_10_gan_models()

In [ ]:
# select GAN model
gan_model_number = min_key

# generate synthetic data using GAN
generator_model_trained = load_model('generator_model_%03d.h5' % gan_model_number)
latent_points = generate_latent_points(latent_dim, len(X_train))
X_synthetic = generator_model_trained.predict(latent_points)

# generate synthetic labels using GAN
discriminator_model_trained = load_model('discriminator_model_%03d.h5' % gan_model_number)
y_synthetic = discriminator_model_trained.predict(X_synthetic)

# clean synthetic data
X_synthetic = scaler.inverse_transform(X_synthetic)
X_synthetic = np.round(X_synthetic)
y_synthetic[y_synthetic<=0.5] = 0
y_synthetic[y_synthetic>0.5] = 1
print(X_synthetic[0:3])
print(y_synthetic[0:3])

# combine data into expanded dataset
X_combined = np.concatenate((X_train, X_synthetic), axis=0)
y_combined = np.concatenate((y_train, y_synthetic), axis=0)

# calculate JS divergence of synthetic data relative to real data
synthetic_train_JS_score = scipy.spatial.distance.jensenshannon(X_synthetic, X_train, base=2)
print('Train JS Score: ' + str(np.round(synthetic_train_JS_score,4)))
print('Avg JS Score: ' + str(round(gmean(synthetic_train_JS_score),4)))

### Neural Network with GAN Data

In [ ]:
# synthetic data = 100% of real data

# build model
nn_model = Sequential()
nn_model.add(Dense(14, activation='LeakyReLU', input_dim=14))
nn_model.add(Dense(32, activation='LeakyReLU'))
nn_model.add(Dense(32, activation='LeakyReLU'))
nn_model.add(Dense(1, activation='sigmoid'))
nn_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['AUC'])

# set parameters
early_stopping_gan = EarlyStopping(monitor='val_loss', mode='min', patience=400, verbose=1)
model_save_gan = ModelCheckpoint('nn_gan_best_model.h5', save_best_only=True, monitor='val_auc', mode='max', verbose=1)

#train GAN neural network model             
nn_gan_model = nn_model.fit(X_combined, y_combined, validation_data=(X_test, y_test), epochs=2000, batch_size=32, verbose=1, callbacks=[early_stopping_gan, model_save_gan])

In [ ]:
# plot learning curves
pyplot.plot(nn_gan_model.history['auc'], label='train')
pyplot.plot(nn_gan_model.history['val_auc'], label='test')
pyplot.legend()

In [ ]:
# load neural network model
nn_gan_model_loaded = load_model('nn_gan_best_model.h5')

# evaluate model on test data (how model would be evaluated at the time)
predictions = nn_gan_model_loaded.predict(X_test)
nn_gan_model_auc_test = roc_auc_score(y_test, predictions)
print("Neural Net GAN Model AUC on Test Data: " + str(round(nn_gan_model_auc_test,4)))

In [ ]:
# synthetic data = 50% of real data

# generate synthetic data using GAN
gan_model_number = min_key
generator_model_trained = load_model('generator_model_%03d.h5' % gan_model_number)
latent_points = generate_latent_points(latent_dim, int(len(X_train)/2))
X_synthetic_half = generator_model_trained.predict(latent_points)

# generate synthetic labels using GAN
discriminator_model_trained = load_model('discriminator_model_%03d.h5' % gan_model_number)
y_synthetic_half = discriminator_model_trained.predict(X_synthetic_half)

# clean synthetic data
X_synthetic_half = scaler.inverse_transform(X_synthetic_half)
X_synthetic_half = np.round(X_synthetic_half)
y_synthetic_half[y_synthetic_half<=0.5] = 0
y_synthetic_half[y_synthetic_half>0.5] = 1

# combine data into expanded dataset
X_combined_half = np.concatenate((X_train, X_synthetic_half), axis=0)
y_combined_half = np.concatenate((y_train, y_synthetic_half), axis=0)

print(X_synthetic_half.shape)
print(y_synthetic_half.shape)

In [ ]:
# synthetic data = 50% of real data

# build model
nn_model = Sequential()
nn_model.add(Dense(14, activation='LeakyReLU', input_dim=14))
nn_model.add(Dense(32, activation='LeakyReLU'))
nn_model.add(Dense(32, activation='LeakyReLU'))
nn_model.add(Dense(1, activation='sigmoid'))
nn_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['AUC'])

# set parameters
early_stopping_gan = EarlyStopping(monitor='val_loss', mode='min', patience=400, verbose=1)
model_save_gan_half = ModelCheckpoint('nn_gan_half_best_model.h5', save_best_only=True, monitor='val_auc', mode='max', verbose=1)

#train GAN neural network model             
nn_gan_model_half = nn_model.fit(X_combined_half, y_combined_half, validation_data=(X_test, y_test), epochs=2000, batch_size=32, verbose=1, callbacks=[early_stopping_gan, model_save_gan_half])

In [ ]:
# plot learning curves
pyplot.plot(nn_gan_model_half.history['auc'], label='train')
pyplot.plot(nn_gan_model_half.history['val_auc'], label='test')
pyplot.legend()

In [ ]:
# load neural network model
nn_gan_model_half_loaded = load_model('nn_gan_half_best_model.h5')

# evaluate model on test data (how model would be evaluated at the time)
predictions = nn_gan_model_half_loaded.predict(X_test)
nn_gan_model_half_auc_test = roc_auc_score(y_test, predictions)
print("Neural Net GAN Model 50% Sample AUC on Test Data: " + str(round(nn_gan_model_half_auc_test,4)))

In [ ]:
# synthetic data = 2x of real data

# generate synthetic data using GAN
gan_model_number = min_key
generator_model_trained = load_model('generator_model_%03d.h5' % gan_model_number)
latent_points = generate_latent_points(latent_dim, len(X_train)*2)
X_synthetic_2x = generator_model_trained.predict(latent_points)

# generate synthetic labels using GAN
discriminator_model_trained = load_model('discriminator_model_%03d.h5' % gan_model_number)
y_synthetic_2x = discriminator_model_trained.predict(X_synthetic_2x)

# clean synthetic data
X_synthetic_2x = scaler.inverse_transform(X_synthetic_2x)
X_synthetic_2x = np.round(X_synthetic_2x)
y_synthetic_2x[y_synthetic_2x<=0.5] = 0
y_synthetic_2x[y_synthetic_2x>0.5] = 1

# combine data into expanded dataset
X_combined_2x = np.concatenate((X_train, X_synthetic_2x), axis=0)
y_combined_2x = np.concatenate((y_train, y_synthetic_2x), axis=0)

print(X_synthetic_2x.shape)
print(y_synthetic_2x.shape)

In [ ]:
# synthetic data = 2x of real data

# build model
nn_model = Sequential()
nn_model.add(Dense(14, activation='LeakyReLU', input_dim=14))
nn_model.add(Dense(32, activation='LeakyReLU'))
nn_model.add(Dense(32, activation='LeakyReLU'))
nn_model.add(Dense(1, activation='sigmoid'))
nn_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['AUC'])

# set parameters
early_stopping_gan = EarlyStopping(monitor='val_loss', mode='min', patience=400, verbose=1)
model_save_gan_2x = ModelCheckpoint('nn_gan_2x_best_model.h5', save_best_only=True, monitor='val_auc', mode='max', verbose=1)

#train GAN neural network model             
nn_gan_model_2x = nn_model.fit(X_combined_2x, y_combined_2x, validation_data=(X_test, y_test), epochs=2000, batch_size=32, verbose=1, callbacks=[early_stopping_gan, model_save_gan_2x])

In [ ]:
# plot learning curves
pyplot.plot(nn_gan_model_2x.history['auc'], label='train')
pyplot.plot(nn_gan_model_2x.history['val_auc'], label='test')
pyplot.legend()

In [ ]:
# load neural network model
nn_gan_model_2x_loaded = load_model('nn_gan_2x_best_model.h5')

# evaluate model on test data (how model would be evaluated at the time)
predictions = nn_gan_model_2x_loaded.predict(X_test)
nn_gan_model_2x_auc_test = roc_auc_score(y_test, predictions)
print("Neural Net GAN Model 2x Sample AUC on Test Data: " + str(round(nn_gan_model_2x_auc_test,4)))

In [ ]:
# synthetic data = 5x of real data

# generate synthetic data using GAN
gan_model_number = min_key
generator_model_trained = load_model('generator_model_%03d.h5' % gan_model_number)
latent_points = generate_latent_points(latent_dim, len(X_train)*5)
X_synthetic_5x = generator_model_trained.predict(latent_points)

# generate synthetic labels using GAN
discriminator_model_trained = load_model('discriminator_model_%03d.h5' % gan_model_number)
y_synthetic_5x = discriminator_model_trained.predict(X_synthetic_5x)

# clean synthetic data
X_synthetic_5x = scaler.inverse_transform(X_synthetic_5x)
X_synthetic_5x = np.round(X_synthetic_5x)
y_synthetic_5x[y_synthetic_5x<=0.5] = 0
y_synthetic_5x[y_synthetic_5x>0.5] = 1

# combine data into expanded dataset
X_combined_5x = np.concatenate((X_train, X_synthetic_5x), axis=0)
y_combined_5x = np.concatenate((y_train, y_synthetic_5x), axis=0)

print(X_synthetic_5x.shape)
print(y_synthetic_5x.shape)

In [ ]:
# synthetic data = 5x of real data

# build model
nn_model = Sequential()
nn_model.add(Dense(14, activation='LeakyReLU', input_dim=14))
nn_model.add(Dense(32, activation='LeakyReLU'))
nn_model.add(Dense(32, activation='LeakyReLU'))
nn_model.add(Dense(1, activation='sigmoid'))
nn_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['AUC'])

# set parameters
early_stopping_gan = EarlyStopping(monitor='val_loss', mode='min', patience=400, verbose=1)
model_save_gan_5x = ModelCheckpoint('nn_gan_5x_best_model.h5', save_best_only=True, monitor='val_auc', mode='max', verbose=1)

#train GAN neural network model             
nn_gan_model_5x = nn_model.fit(X_combined_5x, y_combined_5x, validation_data=(X_test, y_test), epochs=2000, batch_size=32, verbose=1, callbacks=[early_stopping_gan, model_save_gan_5x])

In [ ]:
# plot learning curves
pyplot.plot(nn_gan_model_5x.history['auc'], label='train')
pyplot.plot(nn_gan_model_5x.history['val_auc'], label='test')
pyplot.legend()

In [ ]:
# load neural network model
nn_gan_model_5x_loaded = load_model('nn_gan_5x_best_model.h5')

# evaluate model on test data (how model would be evaluated at the time)
predictions = nn_gan_model_5x_loaded.predict(X_test)
nn_gan_model_5x_auc_test = roc_auc_score(y_test, predictions)
print("Neural Net GAN Model 5x Sample AUC on Test Data: " + str(round(nn_gan_model_5x_auc_test,4)))

In [ ]:
# select neural network model using GAN synthetic data with best test data AUC 
best_gan_version = nn_gan_model_half_loaded

# evaluate best GAN synthetic data model on out of sample data (simulating a production environment)
predictions = best_gan_version.predict(X_oos)
nn_gan_model_auc_oos = roc_auc_score(y_oos, predictions)
print("Neural Net GAN Model AUC on Out of Sample Data: " + str(round(nn_gan_model_auc_oos,4)))

## Diffusion Model

In [ ]:
# normalize training data to 0 mean and unit variance
scaler_2 = StandardScaler()
X_train_adj_2 = scaler_2.fit_transform(X_train)  

# define utility functions
def make_beta_schedule(n_timesteps, schedule, start, end):
    if schedule == 'linear':
        betas = torch.linspace(start, end, n_timesteps)
    elif schedule == "quad":
        betas = torch.linspace(start ** 0.5, end ** 0.5, n_timesteps) ** 2
    elif schedule == "sigmoid":
        betas = torch.linspace(-6, 6, n_timesteps)
        betas = torch.sigmoid(betas) * (end - start) + start
    return betas

def extract(input, t, x):
    shape = x.shape
    out = torch.gather(input, 0, t.to(input.device))
    reshape = [t.shape[0]] + [1] * (len(shape) - 1)
    return out.reshape(*reshape)

# generate synthetic data
def p_sample(model, x, t, alphas, betas, one_minus_alphas_bar_sqrt):
    t = torch.tensor([t])
    # Factor to the model output
    eps_factor = ((1 - extract(alphas, t, x)) / extract(one_minus_alphas_bar_sqrt, t, x))
    # Model output
    eps_theta = model(x, t)
    # Final values
    mean = (1 / extract(alphas, t, x).sqrt()) * (x - (eps_factor * eps_theta))
    # Generate z
    z = torch.randn_like(x)
    # Fixed sigma
    sigma_t = extract(betas, t, x).sqrt()
    sample = mean + sigma_t * z
    return (sample)

def p_sample_loop(model, shape, n_steps, alphas, betas, one_minus_alphas_bar_sqrt):
    cur_x = torch.randn(shape)
    x_seq = [cur_x]
    for i in reversed(range(n_steps)):
        cur_x = p_sample(model, cur_x, i, alphas, betas, one_minus_alphas_bar_sqrt)
        x_seq.append(cur_x)
    return x_seq

# define loss function
def noise_estimation_loss(model, x_0, alphas_bar_sqrt, one_minus_alphas_bar_sqrt, n_steps):
    batch_size = x_0.shape[0]
    # Select a random step for each example
    t = torch.randint(0, n_steps, size=(batch_size // 2 + 1,))
    t = torch.cat([t, n_steps - t - 1], dim=0)[:batch_size].long()
    # x0 multiplier
    a = extract(alphas_bar_sqrt, t, x_0)
    # eps multiplier
    am1 = extract(one_minus_alphas_bar_sqrt, t, x_0)
    e = torch.randn_like(x_0)
    # model input
    x = x_0 * a + e * am1
    output = model(x, t)
    return (e - output).square().mean()

# core diffusion model architecture
class ConditionalLinear(nn.Module):
    def __init__(self, num_in, num_out, n_steps):
        super(ConditionalLinear, self).__init__()
        self.num_out = num_out
        self.lin = nn.Linear(num_in, num_out)
        self.embed = nn.Embedding(n_steps, num_out)
        self.embed.weight.data.uniform_()

    def forward(self, x, y):
        out = self.lin(x)
        gamma = self.embed(y)
        out = gamma.view(-1, self.num_out) * out
        return out
        
class ConditionalModel(nn.Module):
    def __init__(self, n_steps):
        super(ConditionalModel, self).__init__()
        self.lin1 = ConditionalLinear(14, 56, n_steps)
        self.norm1 = nn.GroupNorm(4, 56)
        self.lin2 = ConditionalLinear(56, 44, n_steps)
        self.norm2 = nn.GroupNorm(4, 44)
        self.lin3 = ConditionalLinear(44, 28, n_steps)
        self.norm3 = nn.GroupNorm(4, 28)
        self.lin4 = nn.Linear(28, 14)
        self.drop = nn.Dropout(0.1)
    
    def forward(self, x, y):
        x = F.softplus(self.lin1(x, y))
        x = self.norm1(x)
        x = self.drop(x)
        x = F.softplus(self.lin2(x, y))
        x = self.norm2(x)
        x = self.drop(x)
        x = F.softplus(self.lin3(x, y))
        x = self.norm3(x)
        x = self.drop(x)
        return self.lin4(x)

# exponential moving average
class EMA(object):
    def __init__(self, mu):
        self.mu = mu
        self.shadow = {}

    def register(self, module):
        for name, param in module.named_parameters():
            if param.requires_grad:
                self.shadow[name] = param.data.clone()

    def update(self, module):
        for name, param in module.named_parameters():
            if param.requires_grad:
                self.shadow[name].data = (1. - self.mu) * param.data + self.mu * self.shadow[name].data

    def ema(self, module):
        for name, param in module.named_parameters():
            if param.requires_grad:
                param.data.copy_(self.shadow[name].data)

    def ema_copy(self, module):
        module_copy = type(module)(module.config).to(module.config.device)
        module_copy.load_state_dict(module.state_dict())
        self.ema(module_copy)
        return module_copy

    def state_dict(self):
        return self.shadow

    def load_state_dict(self, state_dict):
        self.shadow = state_dict
        
# set hyperparameters        
epochs = 2001 
n_steps = 100 
batch_size = 64
beta_start = 0.0001
beta_end = 0.02
betas = make_beta_schedule(n_timesteps=n_steps, schedule='linear', start=beta_start, end=beta_end)

alphas = 1 - betas
alphas_prod = torch.cumprod(alphas, 0)
alphas_prod_p = torch.cat([torch.tensor([1]).float(), alphas_prod[:-1]], 0)
alphas_bar_sqrt = torch.sqrt(alphas_prod)
one_minus_alphas_bar_sqrt = torch.sqrt(1 - alphas_prod)

model = ConditionalModel(n_steps)
optimizer = optim.Adam(model.parameters(), lr=0.001)
data = X_train_adj_2.tolist()
dataset = torch.tensor(data).float()
table_dim = 14

# create EMA model
ema = EMA(0.999)
ema.register(model)

# run model
for t in range(epochs):
    permutation = torch.randperm(dataset.size()[0])
    for i in range(0, dataset.size()[0], batch_size):
        # retrieve current batch
        indices = permutation[i:i+batch_size]
        batch_x = dataset[indices]
        # calculate loss and train model
        loss = noise_estimation_loss(model, batch_x, alphas_bar_sqrt, one_minus_alphas_bar_sqrt, n_steps)
        optimizer.zero_grad()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.)
        optimizer.step()
        # update exponential moving average
        ema.update(model)
    # print loss, generate synthetic data, and save outputs
    if (t % 100 == 0):
        print(f"{t+1}/{epochs} Loss: {loss}")
        # to change the amount of synthetic data generated, adjust shape parameter in p_sample_loop
        x_seq = p_sample_loop(model, dataset.shape, n_steps, alphas, betas, one_minus_alphas_bar_sqrt)
        csv_path = 'diffusion_sythetic_data_%d.csv' % (t+1)        
        pd.DataFrame(x_seq[n_steps].detach().numpy()).to_csv(csv_path, header=False, index=False)
        torch.save({'epoch': t+1,
                    'model_state_dict': model.state_dict(),
                    'optimizer_state_dict': optimizer.state_dict(),
                    'loss': loss}, 
                    'diffusion_model_%03d.pth' % (t+1))

In [ ]:
# calculate JS scores for diffusion generated data and identify best diffusion model iteration
diffusion_model_js_scores = {}

def compile_diffusion_model_js_scores(epochs):
    for i in range(epochs):
        if i % 100 == 0 and i > 0:
#            _X_synthetic_diffusion = eval('x_seq_%d' % (i+1))[n_steps]
#            _X_synthetic_diffusion = _X_synthetic_diffusion.detach().numpy()
            _X_synthetic_diffusion = pd.read_csv('diffusion_sythetic_data_%d.csv' % (i+1),header=None)
            _X_synthetic_diffusion = scaler_2.inverse_transform(_X_synthetic_diffusion)
            _X_synthetic_diffusion = tf.clip_by_value(_X_synthetic_diffusion, 0, X_train.max())
            _X_synthetic_diffusion = np.round(_X_synthetic_diffusion)

            # calculate JS scores and append
            synthetic_train_JS_score_diffusion = scipy.spatial.distance.jensenshannon(_X_synthetic_diffusion, X_train, base=2)
            diffusion_model_js_scores[i] = round(gmean(synthetic_train_JS_score_diffusion),4)

compile_diffusion_model_js_scores(epochs)

# return best model iteration
min_value = min(diffusion_model_js_scores.values())
min_key = min(diffusion_model_js_scores, key=diffusion_model_js_scores.get)
print(f"Best Model Iteration: {min_key}, Avg JS Score: {min_value}")

# return best 10 model iterartions
def top_10_diffusion_models():
    print("Top 10 Diffusion Model Iterations")
    top_10_diffusion_models = sorted(diffusion_model_js_scores, key=diffusion_model_js_scores.get, reverse=False)[:10]
    for i in range(10):
        key = top_10_diffusion_models[i]
        value = diffusion_model_js_scores[key]
        print(f"Model Iteration: {key}, Avg JS Score: {value}")

top_10_diffusion_models()

In [ ]:
# select diffusion model
diffusion_model_number = min_key

# generate synthetic data using diffusion
X_synthetic_diffusion = pd.read_csv('diffusion_sythetic_data_%d.csv' % (min_key+1),header=None)
X_synthetic_diffusion = scaler_2.inverse_transform(X_synthetic_diffusion)
X_synthetic_diffusion = tf.clip_by_value(X_synthetic_diffusion, 0, X_train.max())
X_synthetic_diffusion = np.round(X_synthetic_diffusion)


# generate synthetic labels using neural network with no synthetic data
y_synthetic_diffusion = nn_raw_model.predict(X_synthetic_diffusion)
y_synthetic_diffusion[y_synthetic_diffusion<=0.5] = 0
y_synthetic_diffusion[y_synthetic_diffusion>0.5] = 1

print(X_synthetic_diffusion[0:3])
print(y_synthetic_diffusion[0:3])

# combine data into expanded dataset
X_combined_diffusion = np.concatenate((X_train, X_synthetic_diffusion), axis=0)
y_combined_diffusion = np.concatenate((y_train, y_synthetic_diffusion), axis=0)

# calculate JS divergence of synthetic data relative to real data
synthetic_train_JS_score = scipy.spatial.distance.jensenshannon(X_synthetic_diffusion, X_train, base=2)
print('Train JS Score: ' + str(np.round(synthetic_train_JS_score,4)))
print('Avg JS Score: ' + str(round(gmean(synthetic_train_JS_score),4)))

In [ ]:
# generate alternative sized synthetic data sample

# set data scalar
data_scalar = 0.5
data_size = [round(data_scalar * len(X_train)), len(X_train.columns)]

# load diffusion model
model = ConditionalModel(n_steps)
checkpoint = torch.load('diffusion_model_%03d.pth' % (diffusion_model_number+1))
model.load_state_dict(checkpoint['model_state_dict'])

# generate synthetic data
x_seq_scaled = p_sample_loop(model, torch.ones(data_size).shape, n_steps, alphas, betas, one_minus_alphas_bar_sqrt)
X_synthetic_diffusion_scaled = x_seq_scaled[n_steps]
X_synthetic_diffusion_scaled = X_synthetic_diffusion_scaled.detach().numpy()
X_synthetic_diffusion_scaled = scaler_2.inverse_transform(X_synthetic_diffusion_scaled)
X_synthetic_diffusion_scaled = tf.clip_by_value(X_synthetic_diffusion_scaled, 0, X_train.max())
X_synthetic_diffusion_scaled = np.round(X_synthetic_diffusion_scaled)

# generate synthetic labels using neural network with no synthetic data
y_synthetic_diffusion_scaled = nn_raw_model.predict(X_synthetic_diffusion_scaled)
y_synthetic_diffusion_scaled[y_synthetic_diffusion_scaled<=0.5] = 0
y_synthetic_diffusion_scaled[y_synthetic_diffusion_scaled>0.5] = 1

# combine data into expanded dataset
X_combined_diffusion_scaled = np.concatenate((X_train, X_synthetic_diffusion_scaled), axis=0)
y_combined_diffusion_scaled = np.concatenate((y_train, y_synthetic_diffusion_scaled), axis=0)

print(X_combined_diffusion_scaled.shape)
print(y_combined_diffusion_scaled.shape)

### Neural Network with Diffusion Model Data

In [ ]:
# synthetic data = 100% of real data

# build model
nn_model = Sequential()
nn_model.add(Dense(14, activation='LeakyReLU', input_dim=14))
nn_model.add(Dense(32, activation='LeakyReLU'))
nn_model.add(Dense(32, activation='LeakyReLU'))
nn_model.add(Dense(1, activation='sigmoid'))
nn_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['AUC'])

# set parameters
early_stopping_diffusion = EarlyStopping(monitor='val_loss', mode='min', patience=400, verbose=1)
model_save_diffusion = ModelCheckpoint('nn_diffusion_best_model.h5', save_best_only=True, monitor='val_auc', mode='max', verbose=1)

#train diffusion neural network model             
nn_diffusion_model = nn_model.fit(X_combined_diffusion, y_combined_diffusion, validation_data=(X_test, y_test), epochs=2000, batch_size=32, verbose=1, callbacks=[early_stopping_diffusion, model_save_diffusion])

In [ ]:
# plot learning curves
pyplot.plot(nn_diffusion_model.history['auc'], label='train')
pyplot.plot(nn_diffusion_model.history['val_auc'], label='test')
pyplot.legend()

In [ ]:
# load neural network model
nn_diffusion_model_loaded = load_model('nn_diffusion_best_model.h5')

# evaluate model on test data (how model would be evaluated at the time)
predictions = nn_diffusion_model_loaded.predict(X_test)
nn_diffusion_model_auc_test = roc_auc_score(y_test, predictions)
print("Neural Net Diffusion Model AUC on Test Data: " + str(round(nn_diffusion_model_auc_test,4)))

In [ ]:
# synthetic data = 50% of real data

# build model
nn_model = Sequential()
nn_model.add(Dense(14, activation='LeakyReLU', input_dim=14))
nn_model.add(Dense(32, activation='LeakyReLU'))
nn_model.add(Dense(32, activation='LeakyReLU'))
nn_model.add(Dense(1, activation='sigmoid'))
nn_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['AUC'])

# set parameters
early_stopping_diffusion = EarlyStopping(monitor='val_loss', mode='min', patience=400, verbose=1)
model_save_diffusion_half = ModelCheckpoint('nn_diffusion_best_model_half.h5', save_best_only=True, monitor='val_auc', mode='max', verbose=1)

#train GAN neural network model             
nn_diffusion_model_half = nn_model.fit(X_combined_diffusion_scaled, y_combined_diffusion_scaled, validation_data=(X_test, y_test), epochs=2000, batch_size=32, verbose=1, callbacks=[early_stopping_diffusion, model_save_diffusion_half])

In [ ]:
# plot learning curves
pyplot.plot(nn_diffusion_model_half.history['auc'], label='train')
pyplot.plot(nn_diffusion_model_half.history['val_auc'], label='test')
pyplot.legend()

In [ ]:
# load neural network model
nn_diffusion_model_half_loaded = load_model('nn_diffusion_best_model_half.h5')

# evaluate model on test data (how model would be evaluated at the time)
predictions = nn_diffusion_model_half_loaded.predict(X_test)
nn_diffusion_model_auc_test = roc_auc_score(y_test, predictions)
print("Neural Net Diffusion Model 50% Sample AUC on Test Data: " + str(round(nn_diffusion_model_auc_test,4)))

In [ ]:
# synthetic data = 2x of real data

# build model
nn_model = Sequential()
nn_model.add(Dense(14, activation='LeakyReLU', input_dim=14))
nn_model.add(Dense(32, activation='LeakyReLU'))
nn_model.add(Dense(32, activation='LeakyReLU'))
nn_model.add(Dense(1, activation='sigmoid'))
nn_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['AUC'])

# set parameters
early_stopping_diffusion = EarlyStopping(monitor='val_loss', mode='min', patience=400, verbose=1)
model_save_diffusion_2x = ModelCheckpoint('nn_diffusion_best_model_2x.h5', save_best_only=True, monitor='val_auc', mode='max', verbose=1)

#train GAN neural network model             
nn_diffusion_model_2x = nn_model.fit(X_combined_diffusion_scaled, y_combined_diffusion_scaled, validation_data=(X_test, y_test), epochs=2000, batch_size=32, verbose=1, callbacks=[early_stopping_diffusion, model_save_diffusion_2x])

In [ ]:
# plot learning curves
pyplot.plot(nn_diffusion_model_2x.history['auc'], label='train')
pyplot.plot(nn_diffusion_model_2x.history['val_auc'], label='test')
pyplot.legend()

In [ ]:
# load neural network model
nn_diffusion_model_2x_loaded = load_model('nn_diffusion_best_model_2x.h5')

# evaluate model on test data (how model would be evaluated at the time)
predictions = nn_diffusion_model_2x_loaded.predict(X_test)
nn_diffusion_model_auc_test = roc_auc_score(y_test, predictions)
print("Neural Net Diffusion Model 2x Sample AUC on Test Data: " + str(round(nn_diffusion_model_auc_test,4)))

In [ ]:
# select neural network model using diffusion mdel synthetic data with best test data AUC 
best_diffusion_version = nn_diffusion_model_half_loaded

# evaluate model on out of sample data (simulating a production environment)
predictions = best_diffusion_version.predict(X_oos)
nn_diffusion_model_auc_oos = roc_auc_score(y_oos, predictions)
print("Neural Net Diffusion Model AUC on Out of Sample Data: " + str(round(nn_diffusion_model_auc_oos,4)))

## Results Summary

In [ ]:
print("Neural Net with No Synthetic Data: {}".format(round(nn_raw_model_auc_oos,4)))
print("Neural Net with GAN Synthetic Data: {}".format(round(nn_gan_model_auc_oos,4)))
print("Neural Net with Diffusion Synthetic Data: {}".format(round(nn_diffusion_model_auc_oos,4)))